A World folder maintains the following files and subfolders:
* game_settings.json
* zero or more agent folders with name \<agent_name\> of your choice:
    1. \<agent_name\>.pth
    1. fig.png
    1. log.txt
    1. performance.csv
    1. all_rows.json
    1. old_\<agent_name\>.pth [HOLD]

The game_settings defines the game that all the agents will be playing.
It should not be modified once any agent is created.

In [ ]:
import os

import sys
import subprocess
import pkg_resources

import argparse
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy
from datetime import datetime
from pytz import timezone
from collections import OrderedDict
import more_itertools as mit
import pandas as pd
import random
import re # regular expression

from enum import Enum 

from traitlets.traitlets import HasTraits, Int, Unicode, default

from typing import List, Set, Dict, Tuple, Any

import importlib

# import Holoviz panel

In [ ]:
if 'panel' not in [pkg.key for pkg in pkg_resources.working_set]:
    !pip install panel
else:
    print('panel already installed')

from bokeh.core.validation import check

import panel as pn
from panel.interact import interact, fixed
from panel import widgets
import param

# import rlcard

In [ ]:
if 'rlcard' not in [pkg.key for pkg in pkg_resources.working_set]:
    !pip install rlcard
else:
    print('rlcard already installed')

In [ ]:
import rlcard
from rlcard.agents import RandomAgent
from rlcard.utils import get_device, set_seed, tournament, reorganize, Logger, plot_curve
from rlcard.agents import DQNAgent
from rlcard.models.gin_rummy_rule_models import GinRummyNoviceRuleAgent

from rlcard.agents.dqn_agent import Memory
from rlcard.agents.dqn_agent import Estimator

from rlcard.games.gin_rummy.player import GinRummyPlayer
from rlcard.games.gin_rummy.game import GinRummyGame
from rlcard.games.gin_rummy.utils.action_event import ActionEvent, DiscardAction
from rlcard.games.gin_rummy.utils.action_event import KnockAction, GinAction, DeclareDeadHandAction

from rlcard.games.gin_rummy.utils.scorers import GinRummyScorer
from rlcard.games.gin_rummy.utils.settings import Setting, Settings, DealerForRound

from rlcard.games.gin_rummy.utils.action_event import draw_card_action_id, pick_up_discard_action_id

import rlcard.games.gin_rummy.utils.utils as utils
import rlcard.games.gin_rummy.utils.melding as melding

from rlcard.games.gin_rummy.utils.thinker import Thinker

from rlcard.games.base import Card

from rlcard.envs.gin_rummy import GinRummyEnv

from rlcard.agents import NFSPAgent

import rlcard as rlcard # NOTE: don't know why this is needed; don't know why it has to be placed last

# Import from gin_rummy_lib

In [ ]:
!git clone https://github.com/billh0420/panel-demo.git

In [ ]:
if not '/content/panel-demo/Gin_Rummy_World/gin_rummy_lib' in sys.path:
    sys.path.append('/content/panel-demo/Gin_Rummy_World/gin_rummy_lib')
from util import to_int_list
from util import sortByRankBySuit
from DQNAgentConfig import DQNAgentConfig
from pane.GameSettingsPane import GameSettingsPane

from util import to_int_list
from util import sortByRankBySuit
from pane.GameSettingsPane import GameSettingsPane
from DQNAgentConfig import DQNAgentConfig
from RLTrainerConfig import RLTrainerConfig
from RLTrainer import RLTrainer
from GinRummyScorer230402 import GinRummyScorer230402
from GameObserver import GameObserver
from GinRummyRookie01RuleAgent import GinRummyRookie01RuleAgent
from GameObserver import GameObserver
from GameReviewer import GameReviewer
from World import World
from pane.DQNAgentPane import DQNAgentPane
from pane.TrainerPane import TrainerPane
from pane.ReviewPlayWindow import ReviewPlayWindow
from pane.WorldGameSettingsView import WorldGameSettingsView
from pane.WorldDQNAgentSettingsView import WorldDQNAgentSettingsView
from pane.WorldRLTrainerSettingsView import WorldRLTrainerSettingsView
from pane.TrainingResultsWindow import TrainingResultsWindow
from pane.ReviewMatchGamesWindow import ReviewMatchGamesWindow

from util import get_current_time

In [ ]:
pn.extension(raw_css=["""
div.orange_border_table + table * {
  border: 1px solid orange;
}
"""])

css_log_widget_box = '''
.bk.log-widget-box {
  color: #ffffff;
  background: #000000;
  border: 1px black solid;
}
'''
pn.extension(raw_css=[css_log_widget_box])

pn.extension('tabulator')

# Change current directory to Gin_Rummy_World

In [ ]:
%cd panel-demo/Gin_Rummy_World

# Final Version (Keep) 230407

In [ ]:
# Step 1 (one time only)
# Create an empty directory for your world.
# For example, create a directory with the name "Gin_Rummy_World"
# This directory will hold all the files that are used in your training sessions.
# This is done one time only.

In [ ]:
# Step 2 (always done to get handle on the world with your choice of name.)

# Open world.
#
# This is done each time you want to do something with your agents.
# Here the world is called 'gin_rummy_world'. You can call it simply 'world' if you wish.
#
# If you do not specify the world_dir, then the current working directory is used.
# If you specify its directory, then it must exists else a crash will result.
# Example: gin_rummy_world = World(world_dir='../results_gin_rummy_dqn')
#
# Note: The world variable is the only variable that is allocated in these steps.
#       I am trying to minimize the creation of other named variables (e.g. views and windows).
#
# The world_dir will hold the following:
#   1) The file game_settings.json (It will be automatically created in step 3 if it doesn't exist).
#   2) The directory whose name is the agent name (It will be automatically created when the agent is created).
#       This directory holds the following files:
#       a) the agent pth file (modified by training it in step 6);
#       b) the file all_rows.json used by the ShowReviewMatchGamesWindow;
#       c) the file log.txt created by play_train_match in step 6;
#       d) the file fig.png created by play_train_match in step 6;
#       e) the file performance.csv created by play_train_match in step 6.

gin_rummy_world = World() # Using current directory as world_dir

In [ ]:
# Step 3 (optional)
# Use WorldGameSettingsView to configure game settings (one time only)
# After you create your first training agent in the next step, you should not change the game settings.
# The agent assumes that it is playing with the game settings that existed when it was created.

WorldGameSettingsView(world=gin_rummy_world).view

In [ ]:
# Step 4 (optional)
# Use WorldDQNAgentSettingsView to create your training agent (one time only)
# You adjust the settings and hit the "Create DQN Agent" button.
# If an agent with the chosen name already exists.
# You will get a message on whether you were successful or not (not implemented).

WorldDQNAgentSettingsView(world=gin_rummy_world).view

In [ ]:
# Step 5 (optional)
# Use WorldRLTrainerSettingsView to set trainer settings (multiple times except for changing algorithm)

WorldRLTrainerSettingsView(world=gin_rummy_world).view

In [ ]:
# Step 6 (optional)
# Run training session (long)

gin_rummy_world.play_train_match(num_episodes=10)

In [ ]:
# Step 7 (optional)
# Use TrainingResultsWindow to see results of training (optional)
# Repeat steps 5, 6, 7 as often as you want. Of course, step 5 and 7 will be optional.

TrainingResultsWindow(world=gin_rummy_world)

In [ ]:
# Step 8 (optional)
# Use ReviewMatchGamesWindow to review how well the dqn_agent is doing (optional)
# Note that 0 <= max_review_episodes <= 1000.
# Repeat steps 5 trough 8.

ReviewMatchGamesWindow(world=gin_rummy_world)